# 导入模块

In [2]:
import os
os.chdir("..")
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize

# 数据准备

In [3]:
# 所有 ETF
tickers = ["EEM","EFA","JPXN","SPY","XLK",'VTI','AGG','DBC']
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1/n_assets)  # 每个资产的权重都是 1/N

# 读取 log return
return_df = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df = pd.concat([return_df, df], axis=1)

# 定义每个月初作为投资起始点
start_date = pd.to_datetime("2024-01-01")
months = 12

# 等权投资

In [ ]:
# 所有 ETF
tickers = ["EEM","EFA","JPXN","SPY","XLK",'VTI','AGG','DBC']
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1/n_assets)  # 每个资产的权重都是 1/N

# 读取 log return
return_df = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df = pd.concat([return_df, df], axis=1)

# 定义每个月初作为投资起始点
start_date = pd.to_datetime("2024-01-01")
months = 12

results = []

for i in range(months):
    infer_start = start_date + relativedelta(months=i)
    infer_end = (infer_start + relativedelta(months=1)) - pd.Timedelta(days=1)

    try:
        # 获取这个月每天的算术收益率
        arith_return_month = np.expm1(return_df.loc[infer_start:infer_end, tickers].values)
        daily_return = arith_return_month @ equal_weight  # 每日组合收益
        monthly_return = np.prod(1 + daily_return) - 1     # 月组合收益

    except Exception as e:
        print(f"⚠️ 无法计算 {infer_start.strftime('%Y-%m')} 的组合收益：{e}")
        monthly_return = np.nan

    print(f"📅 {infer_start.strftime('%Y-%m')} 等权组合收益：{monthly_return:.4%}")
    results.append((infer_start.strftime('%Y-%m'), monthly_return))

# 保存结果
df_result = pd.DataFrame(results, columns=["Month", "MonthlyReturn"])
df_result["CumulativeReturn"] = (1 + df_result["MonthlyReturn"]).cumprod() - 1
df_result.to_csv("result\8_ticker_1ytrain1yinfer\EqualWeight.csv", index=False)


📅 2024-01 等权组合收益：0.5294%
📅 2024-02 等权组合收益：2.9386%
📅 2024-03 等权组合收益：2.7769%
📅 2024-04 等权组合收益：-2.9288%
📅 2024-05 等权组合收益：3.4828%
📅 2024-06 等权组合收益：1.9637%
📅 2024-07 等权组合收益：0.9891%
📅 2024-08 等权组合收益：1.3077%
📅 2024-09 等权组合收益：1.9309%
📅 2024-10 等权组合收益：-2.2028%
📅 2024-11 等权组合收益：1.9947%
📅 2024-12 等权组合收益：-1.6464%


# 最大化夏普比率

## 夏普比率优化函数

In [4]:
def max_sharpe_portfolio(mu, cov, risk_free=0.0):
    n = len(mu)

    def neg_sharpe(w):
        port_return = np.dot(w, mu)
        port_vol = np.sqrt(np.dot(w, np.dot(cov, w)))
        return -((port_return - risk_free) / port_vol)

    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })
    bounds = [(0.0, 1.0)] * n
    w0 = np.array([1.0 / n] * n)

    result = minimize(neg_sharpe, w0, bounds=bounds, constraints=constraints)
    return result.x  # 最优权重


## 月度回测代码

In [6]:
results = []

for i in range(12):
    infer_start = start_date + relativedelta(months=i)
    infer_end = (infer_start + relativedelta(months=1)) - pd.Timedelta(days=1)
    train_start = infer_start - relativedelta(years=1)
    train_end = infer_start - pd.Timedelta(days=1)

    try:
        # 1. 获取训练区间的算术收益率
        train_data = return_df.loc[train_start:train_end, tickers]
        train_return = np.expm1(train_data)
        daily_mu = train_return.mean()
        daily_cov = train_return.cov()

        # 2. 年化收益与协方差
        mu_annual = daily_mu * 252
        cov_annual = daily_cov * 252

        # 3. 求最大夏普比组合
        weights = max_sharpe_portfolio(mu_annual.values, cov_annual.values)

        # 4. 用该组合评估当前月收益
        test_return = np.expm1(return_df.loc[infer_start:infer_end, tickers].values)
        daily_return = test_return @ weights
        monthly_return = np.prod(1 + daily_return) - 1

    except Exception as e:
        print(f"⚠️ {infer_start.strftime('%Y-%m')} 出错：{e}")
        monthly_return = np.nan
        weights = [np.nan] * len(tickers)

    print(f"📅 {infer_start.strftime('%Y-%m')} Max Sharpe 月收益：{monthly_return:.4%}")
    results.append((infer_start.strftime('%Y-%m'), monthly_return))

df_maxsharpe = pd.DataFrame(results, columns=["Month", "MonthlyReturn"])
df_maxsharpe["CumulativeReturn"] = (1 + df_maxsharpe["MonthlyReturn"]).cumprod() - 1
df_maxsharpe.to_csv("result/8_ticker_1ytrain1yinfer/MaxSharpe.csv", index=False)


📅 2024-01 Max Sharpe 月收益：1.9125%
📅 2024-02 Max Sharpe 月收益：4.7046%
📅 2024-03 Max Sharpe 月收益：1.2596%
📅 2024-04 Max Sharpe 月收益：-4.1104%
📅 2024-05 Max Sharpe 月收益：4.9560%
📅 2024-06 Max Sharpe 月收益：2.9027%
📅 2024-07 Max Sharpe 月收益：0.6880%
📅 2024-08 Max Sharpe 月收益：2.2207%
📅 2024-09 Max Sharpe 月收益：1.8203%
📅 2024-10 Max Sharpe 月收益：-1.7132%
📅 2024-11 Max Sharpe 月收益：3.5834%
📅 2024-12 Max Sharpe 月收益：-2.0796%
